In [32]:
import pandas as pd
import torch
import copy
import time
from pathlib import Path
from torch import nn
from torch.utils.data import random_split
from torch.utils.tensorboard import SummaryWriter
from sound_classification_dataset import SoundDS
from sound_classification_model import AudioClassifier

In [33]:
# ----------------------------
# Prepare training data from Metadata file
# ----------------------------

data_path = 'UrbanSound8k'

# Read metadata file
metadata_file = data_path + '/UrbanSound8K.csv'
df = pd.read_csv(metadata_file)
df.head()

# Construct file path by concatenating fold and file name
df['relative_path'] = '/fold' + df['fold'].astype(str) + '/' + df['slice_file_name'].astype(str)

# Take relevant columns
# df = df[['relative_path', 'classID']]
df.head()

,slice_file_name,fsID,start,end,salience,fold,classID,class,relative_path
0,100032-3-0-0.wav,100032,0.0,0.317551,1,5,3,dog_bark,/fold5/100032-3-0-0.wav
1,100263-2-0-117.wav,100263,58.5,62.500000,1,5,2,children_playing,/fold5/100263-2-0-117.wav
2,100263-2-0-121.wav,100263,60.5,64.500000,1,5,2,children_playing,/fold5/100263-2-0-121.wav
3,100263-2-0-126.wav,100263,63.0,67.000000,1,5,2,children_playing,/fold5/100263-2-0-126.wav
4,100263-2-0-137.wav,100263,68.5,72.500000,1,5,2,children_playing,/fold5/100263-2-0-137.wav


In [35]:
df.shape

(8732, 9)

In [36]:
writer = SummaryWriter()
myds = SoundDS(df, data_path)

In [37]:
# Random split of 80:20 between training and validation

# dataloaders = {x: torch.utils.data.DataLoader(myds[x], batch_size=4, shuffle=True, num_workers=4) for x in ['train', 'val']}
# dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
# class_names = image_datasets['train'].classes


num_items = len(myds)
num_train = round(num_items * 0.8)
num_val_test = num_items - num_train
train_ds, val_test_ds = random_split(myds, [num_train, num_val_test])
num_val = round(num_val_test/2)
num_test = num_val_test - num_val
val_ds, test_ds = random_split(val_test_ds, [num_val, num_test])

# # Create training and validation data loaders
train_dl = torch.utils.data.DataLoader(train_ds, batch_size=128, shuffle=True)
val_dl = torch.utils.data.DataLoader(val_ds, batch_size=128, shuffle=False)
test_dl = torch.utils.data.DataLoader(test_ds, batch_size=128, shuffle=False)

In [38]:
len(train_dl)

55

In [19]:
# Create the model and put it on the GPU if available
model = nn.DataParallel(AudioClassifier())
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Check that it is on Cuda
next(model.parameters()).device

device(type='cuda', index=0)

In [20]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch + 1, num_epochs))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
                dataloader = train_dl
                data_size = len(train_ds)
            else:
                model.eval()   # Set model to evaluate mode
                dataloader = val_dl
                data_size = len(val_ds)

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloader:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / data_size
            epoch_acc = running_corrects.double() / data_size

            print('{} - Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
                torch.save(best_model_wts, 'model.pt')

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

In [21]:
num_epochs = 100
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=0.001)
scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=0.001,
                                                    steps_per_epoch=int(len(train_dl)),
                                                    epochs=num_epochs,
                                                    anneal_strategy='linear')
    
model_ft = train_model(model, criterion, optimizer, scheduler,
                       num_epochs=num_epochs)
# writer.flush()
# writer.close()

Epoch 1/100
----------
train - Loss: 2.2910 Acc: 0.1420
val - Loss: 2.2332 Acc: 0.2050

Epoch 2/100
----------
train - Loss: 2.2057 Acc: 0.2166
val - Loss: 2.1659 Acc: 0.2703

Epoch 3/100
----------
train - Loss: 2.1411 Acc: 0.2562
val - Loss: 2.1126 Acc: 0.2818

Epoch 4/100
----------
train - Loss: 2.0901 Acc: 0.2738
val - Loss: 2.0709 Acc: 0.3013

Epoch 5/100
----------
train - Loss: 2.0503 Acc: 0.2956
val - Loss: 2.0232 Acc: 0.3196

Epoch 6/100
----------
train - Loss: 2.0167 Acc: 0.3023
val - Loss: 1.9971 Acc: 0.3276

Epoch 7/100
----------
train - Loss: 1.9821 Acc: 0.3119
val - Loss: 1.9719 Acc: 0.3265

Epoch 8/100
----------
train - Loss: 1.9564 Acc: 0.3222
val - Loss: 1.9475 Acc: 0.3597

Epoch 9/100
----------
train - Loss: 1.9327 Acc: 0.3277
val - Loss: 1.9233 Acc: 0.3471

Epoch 10/100
----------
train - Loss: 1.9108 Acc: 0.3352
val - Loss: 1.9021 Acc: 0.3505

Epoch 11/100
----------
train - Loss: 1.8917 Acc: 0.3470
val - Loss: 1.8866 Acc: 0.3608

Epoch 12/100
----------
train 

train - Loss: 1.0457 Acc: 0.6489
val - Loss: 1.0629 Acc: 0.6483

Epoch 94/100
----------
train - Loss: 1.0427 Acc: 0.6577
val - Loss: 1.0706 Acc: 0.6586

Epoch 95/100
----------
train - Loss: 1.0529 Acc: 0.6504
val - Loss: 1.0498 Acc: 0.6564

Epoch 96/100
----------
train - Loss: 1.0435 Acc: 0.6522
val - Loss: 1.0583 Acc: 0.6541

Epoch 97/100
----------
train - Loss: 1.0365 Acc: 0.6600
val - Loss: 1.0571 Acc: 0.6495

Epoch 98/100
----------
train - Loss: 1.0330 Acc: 0.6543
val - Loss: 1.0745 Acc: 0.6518

Epoch 99/100
----------
train - Loss: 1.0356 Acc: 0.6539
val - Loss: 1.0547 Acc: 0.6609

Epoch 100/100
----------
train - Loss: 1.0175 Acc: 0.6590
val - Loss: 1.0571 Acc: 0.6392

Training complete in 204m 6s
Best val Acc: 0.660939


In [46]:
# ----------------------------
# Inference
# ----------------------------
def inference (model, test_dl):
    correct_prediction = 0
    total_prediction = 0

    # Disable gradient updates
    with torch.no_grad():
        for data in test_dl:
            # Get the input features and target labels, and put them on the GPU
            inputs, labels = data[0].to(device), data[1].to(device)

            # Normalize the inputs
            inputs_m, inputs_s = inputs.mean(), inputs.std()
            inputs = (inputs - inputs_m) / inputs_s

            # Get predictions
            outputs = model(inputs)

            # Get the predicted class with the highest score
            _, prediction = torch.max(outputs,1)
            # Count of predictions that matched the target label
            correct_prediction += (prediction == labels).sum()
            print(correct_prediction)
#             print(prediction.shape)
            total_prediction += prediction.shape[0]
#             print(total_prediction)
        
    acc = correct_prediction/total_prediction
    print(f'Accuracy: {acc:.2f}, Total items: {total_prediction}')

In [40]:
len(test_dl)

7

In [47]:
# Run inference on trained model with the validation set load best model weights
model_inf = nn.DataParallel(AudioClassifier())
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_inf = model_inf.to(device)
model_inf.load_state_dict(torch.load('model.pt'))
model_inf.eval()
inference(model_inf, test_dl)

tensor(11, device='cuda:0')
tensor(26, device='cuda:0')
tensor(41, device='cuda:0')
tensor(52, device='cuda:0')
tensor(67, device='cuda:0')
tensor(79, device='cuda:0')
tensor(91, device='cuda:0')
Accuracy: 0.10, Total items: 873


In [27]:
# Visualizing the model predictions
# Generic function to display predictions for a few images

def visualize_model(model, num_images=6):
    was_training = model.training
    model.eval()
    images_so_far = 0
    fig = plt.figure()

    with torch.no_grad():
        for i, (inputs, labels) in enumerate(dataloaders['val']):
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            for j in range(inputs.size()[0]):
                images_so_far += 1
                ax = plt.subplot(num_images//2, 2, images_so_far)
                ax.axis('off')
                ax.set_title('predicted: {}'.format(class_names[preds[j]]))
                imshow(inputs.cpu().data[j])

                if images_so_far == num_images:
                    model.train(mode=was_training)
                    return
        model.train(mode=was_training)